In [ ]:
import csv
import pandas as pd

In [ ]:
with open("/Users/beidan/RASHIP/AutoPhrase/models/DBLP/pattern_score_details_iter1.txt", newline = '') as games:                                                                                          
        game_reader = pd.read_table(games, sep='\t')

In [ ]:
# with open("/Users/beidan/RASHIP/AutoPhrase/tmp/pattern_score_details_1.txt", newline = '') as games:                                                                                          
#         game_reader = pd.read_table(games, sep='\t')

In [ ]:
game_reader = game_reader[:-1]

In [ ]:
game_reader.head()

In [ ]:
game_reader[game_reader['pattern'].str.contains("tricare")].sort_values('final_score', ascending=False)

In [ ]:
test = {"a":1}
if test:
    print(1)
else:
    print(0)

## Custom Patricia class

## My custom tree

In [ ]:
class Word:
    def __init__(self, token, score, stopW = False):
        self.token = token
        self.count = 1
        self.qualitylist = [score]
        self.isStopword = stopW
        
from collections import defaultdict
from tqdm import tqdm
import statistics
import operator
import nltk
from nltk.corpus import stopwords
import math
#nltk.download('stopwords')

In [ ]:
class patricia():
    def __init__(self):
        self._data = {}
        self._tokenMap = {}
    
    def incr(self, word):
        tokenMap = self._tokenMap
        if word.token in tokenMap:
            tokenMap[word.token].count += 1
            tokenMap[word.token].qualitylist += word.qualitylist
        else:
            tokenMap[word.token] = word

    def addWord(self, word):
        self.incr(word)
        
        data = self._data
        i = 0
        while 1:
            try:
                # try this char by char
                node = data[word.token[i:i+1]]
            
            # do not have the key
            except KeyError:
                
                
                # if the dict is not empty
                if data:
                    # initialize a new branch
                    data[word.token[i:i+1]] = [word.token[i+1:],{}]
                    
                else:
                    # if end of word - return
                    if word.token[i:i+1] == '':
                        return
                    else:
                        # if it is not the first character
                        if i != 0:
                            data[''] = ['',{}]
                        # put the rest of words in
                        data[word.token[i:i+1]] = [word.token[i+1:],{}]
                return

            # have the key
            i += 1
            
            # if the word starts with the prefix starting at position i
            if word.token.startswith(node[0],i):
                
                
                # if the trie always has the prefix tree
                #e.g. tricare --> i=1 -> node[0] icare  word[i:] = icare
                if len(word.token[i:]) == len(node[0]):
                    #print ("word {}, i {}, node[0] {}", word, i, node[0])  
                     
                    # if the trie has right
                    if node[1]:
                        try:
                            node[1]['']
                        except KeyError:
                            data = node[1]
                            data[''] = ['',{}]
                    return
                # node[0] is the current prefix
                else:
                    
                    # directly advance
                    i += len(node[0])
                    data = node[1]
            
            else:
                ii = i
                j = 0
                while ii != len(word.token) and j != len(node[0]) and \
                      word.token[ii:ii+1] == node[0][j:j+1]:
                    ii += 1
                    j += 1
                tmpdata = {}
                tmpdata[node[0][j:j+1]] = [node[0][j+1:],node[1]]
                tmpdata[word.token[ii:ii+1]] = [word.token[ii+1:],{}]
                
                #print("tempdata, nord[0], word_part: ", str(tmpdata), node[0],word[ii+1:] )
                data[word.token[i-1:i]] = [node[0][:j],tmpdata]
                return
            
    def isPrefix(self,word):
        data = self._data
        i = 0
        wordlen = len(word.token)
        while 1:
            try:
                node = data[word.token[i:i+1]]
            except KeyError:
                return False
            i += 1
            if word.token.startswith(node[0][:wordlen-i],i):
                if wordlen - i > len(node[0]):
                    i += len(node[0])
                    data = node[1]
                else:
                    return True
            else:
                return False
            
    def isWord(self,word):
        data = self._data
        i = 0
        while 1:
            try:
                node = data[word.token[i:i+1]]
            except KeyError:
                return False
            i += 1
            if word.token.startswith(node[0],i):
                if len(word.token[i:]) == len(node[0]):
                    if node[1]:
                        try:
                            node[1]['']
                        except KeyError:
                            return False
                    return True
                else:
                    i += len(node[0])
                    data = node[1]
            else:
                return False
            
    def printCount(self):
        info = {}
        tokenmap = self._tokenMap
        for token in tokenmap:
            info[token] = tokenmap[token].count
        print(info)
    
    def getVarAll(self):
        info = {}
        tokenmap = self._tokenMap
        for token in tokenmap:
            if token not in set(stopwords.words('english')):
                ls = tokenmap[token].qualitylist
                if (len(ls) < 2):
                        info[token] = 0
                else:
                        info[token] = statistics.variance(ls) * math.log(len(ls))

        return (info)
    


        

  

## construct a list of Word and rank it by variance

In [ ]:


# initialize
dt = defaultdict()
xxx = patricia()
for index, row in tqdm(game_reader.iterrows()):
        pattern = row['pattern']
        tokens = pattern.split(' ')
        score = row['final_score']
        for token in tokens:
            xxx.addWord(Word(token, score))

## now we save all the data in xxx
## add counter to list of Words

In [ ]:
# xxx.printCount()
varsDict = xxx.getVarAll()

In [ ]:
sorted_vars = sorted(varsDict.items(), key=operator.itemgetter(1))
sorted_vars.reverse()

In [ ]:
## write out to file
with open('tmp/sorted_vars.txt', 'w') as f:
    for item in sorted_vars:
        f.write("%s\n" % str(item))

In [ ]:
## find all patterns that contain the words
## this should be retrived from the table
## e.g. game_reader[game_reader['pattern'].str.contains("tricare")].sort_values('final_score', ascending=False)

In [ ]:
sorted_vars[:300]

In [ ]:
ls = game_reader[game_reader['pattern'].str.contains("tricare")].sort_values('final_score', ascending=False).head()
ls['pattern']

In [ ]:
tmp = game_reader[game_reader['pattern'].str.contains(r"\b" + "tricare" + r"\b", regex=True)]
tmp.final_score.plot.kde()

In [ ]:
from scipy import stats
density = stats.gaussian_kde(tmp.final_score.values)
ys = density(tmp.final_score.values)
bb = np.argmax(ys * -1)
aa = ys[bb]

In [ ]:
ys

In [ ]:
valley_indexes = signal.find_peaks(ys * -1)
valley_indexes = valley_indexes[0]

In [ ]:
valley_indexes = signal.argrelmin(ys)
valley_indexes = valley_indexes[0]

In [ ]:
valley_indexes

In [ ]:
ys[valley_indexes]

### Get patterns with high variance

In [ ]:
output = {}
for entry in sorted_vars:
    if entry[1]>0:
        token = entry[0]
        tmp = game_reader[game_reader['pattern'].str.contains(r"\b" + token + r"\b", regex=True)]
        tmp_ls = tmp['pattern'].values.tolist()
        tmp_ls.append(str(entry[1]))
        output[token] = tmp_ls

In [ ]:
output

In [ ]:
## more specific version
output = {}
for entry in sorted_vars[:300]:
    if entry[1]>0:
        token = entry[0]
        tmp = game_reader[game_reader['pattern'].str.contains(r"\b" + token + r"\b", regex=True)]
        tmp_ls = []
        for index, row in tmp.iterrows():
            tmp_ls.append(row['pattern'] + ' ' + str(row['final_score']))
        tmp_ls.append(str(entry[1]))
        output[token] = tmp_ls

In [ ]:
output

In [ ]:
## write the output to file
file1 = open("tmp/522highvarall403.txt","w") 
file1.write(json.dumps(output, indent=2, sort_keys=False))
file1.close()

### Get out of sample patterns

In [ ]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
## e.g.
tmp = game_reader[game_reader['pattern'].str.contains(r"\b" + "president" + r"\b", regex=True)]
[avg, low, high] =mean_confidence_interval(tmp.final_score.tolist())
avg

In [ ]:
import matplotlib
matplotlib.use('Agg') # Bypass the need to install Tkinter GUI framework
 
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_x = np.arange(start = 0, stop = 25, step = 1, dtype='int')
data_y = np.random.random(25)*6

In [ ]:
valley_indexes = signal.argrelextrema(data_y, np.less)
valley_indexes = valley_indexes[0]

In [ ]:
data_y

In [ ]:
valley_indexes

In [ ]:
data = np.array(tmp.final_score.tolist())

In [ ]:
data = [t * 10 for t in data]
data = np.array(data)

In [ ]:
data

In [ ]:
# import KMeans
from sklearn.cluster import KMeans

In [ ]:
# create kmeans object
kmeans = KMeans(n_clusters=3)

In [ ]:
# fit kmeans object to data
kmeans.fit(data)

In [ ]:
valley_indexes = signal.find_peaks_cwt(data * -1, widths)

In [ ]:
valley_indexes

In [ ]:
data[valley_indexes]

In [ ]:
valley_indexes = signal.argrelextrema(data, np.less)
valley_indexes = valley_indexes[0]

In [ ]:
valley_indexes

In [ ]:
(fig, ax) = plt.subplots()

valley_x = valley_indexes
valley_y = tmp.values[valley_indexes]
ax.plot(valley_x, valley_y, marker='o', linestyle='dashed', color='red', label="Valleys")
 

In [ ]:
## iterate through sorted_Values
## count > 30 + out of range

output = {}
for entry in sorted_vars[:30]:
    token = entry[0]
    tmp = game_reader[game_reader['pattern'].str.contains(r"\b" + token + r"\b", regex=True)]
    [avg, low, high] = mean_confidence_interval(tmp.final_score.tolist(), confidence=0.99)
    
    tmp_ls = []
    for index, row in tmp.iterrows():
        if float(row['final_score']) < low or float(row['final_score']) > high:
            tmp_ls.append(row['pattern'] + ' ' + str(row['final_score']))
            print(low, " ", row['final_score'], " ", token, " ", high)
        else:
            print("skip")
    
    if len(tmp_ls) > 0:   
        output[token] = tmp_ls

In [ ]:
output